# XOX PnF charts
## Development Notes

Two methods for populating charts:

- **High and Low** price: whenever H and L are available
- **Last/Close** price: for illiquid assets (or indices). Also for intraday trading using realtime data

Scale types:

- Constant size boxes
- Variable size boxes

In [1]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.2
numpy version: 1.22.3


In [222]:
# Reading a sample data file

data = pd.read_csv("data/QQQ.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-01-03,399.049988,401.940002,396.880005,401.679993,401.184021,40575900
1,2022-01-04,402.239990,402.279999,393.290009,396.470001,395.980438,58027200
2,2022-01-05,394.739990,395.890015,384.029999,384.290009,383.815491,75739800
3,2022-01-06,382.420013,387.350006,380.130005,384.019989,383.545807,70814300
4,2022-01-07,384.029999,385.760010,378.040009,379.859985,379.390961,72652300


In [223]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,102.000000,102.000000,102.000000,102.000000,102.000000,1.020000e+02
mean,342.218922,346.291178,337.129804,341.665784,341.438495,8.282852e+07
std,26.934540,26.199942,26.931199,26.659522,26.542613,2.637948e+07
min,285.709991,288.859985,280.209991,287.239990,287.239990,4.057590e+07
25%,323.810005,329.997498,318.502510,324.994987,324.694145,6.491755e+07
50%,345.835007,348.565002,340.004989,345.279999,344.853668,7.613600e+07
75%,359.159996,362.117493,353.579994,359.292503,359.181648,9.183515e+07
max,402.239990,402.279999,396.880005,401.679993,401.184021,1.986858e+08


In [224]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       102 non-null    object 
 1   Open       102 non-null    float64
 2   High       102 non-null    float64
 3   Low        102 non-null    float64
 4   Close      102 non-null    float64
 5   Adj Close  102 non-null    float64
 6   Volume     102 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 5.7+ KB


In [225]:
# Using only H, L and C data
price_data = data[['High','Low','Close']]

In [226]:
price_data.head()

,High,Low,Close
0,401.940002,396.880005,401.679993
1,402.279999,393.290009,396.470001
2,395.890015,384.029999,384.290009
3,387.350006,380.130005,384.019989
4,385.760010,378.040009,379.859985


## PnF functions

- init_pnf() initializes the first column
- update_pnf() deals with the rest of the chart
- chart_text() generates a string with a text version of the chart

In [189]:
def generate_column_range(scale, range_low, range_high):
    col_range = scale[np.logical_and(scale>=range_low, scale<=range_high)]
    
    return col_range  

In [190]:
def generate_scale(start, end, box_size=1, method='standard'):
    scale = np.arange(start=start, stop=high+box_size, step=box_size)
    
    return scale

In [191]:
def init_pnf(scale,
             high,
             low,
             close,
             reversal_size,
             box_range=[]):
    '''
    returns status as num value and box_range as array
    
    Args:
        scale: np.array - the scale for the chart
        high: float - high price
        low: float  - low price
        close: float - closing/last price
        reversal_size: int > 0
        box_range: np.array or empty list
        
    Returns:
        status: int (-1,0, or 1)
        box_range: np.array
    '''
    
    if len(box_range) == 0:
        box_range = scale[np.logical_and(scale>=low, scale<=high)]
    else:
#         if high > box_range.max():
#             box_range = scale[np.logical_and(scale>=box_range.min(), scale<=high)]
#         if low < box_range.min():
#             box_range = scale[np.logical_and(scale<=box_range.max(), scale>=low)]
        box_range = scale[np.logical_and(scale>=min(box_range.min(), low), scale<=max(box_range.max(), high))]
#         box_range = generate_column_range()
        
    # mid-price defined as the middle of the range
#     mid_price = box_range.min() + (box_range.max() - box_range.min())/2
    mid_price = 0.5 * box_range.min() + 0.5 * box_range.max()

    if len(box_range) >= reversal_size and close > mid_price:
        status = 1
    elif len(box_range) >= reversal_size and close < mid_price:
        status = -1
    else:
        status = 0
        
    return status, box_range

In [192]:
def update_pnf(scale,
               high,
               low,
               status,
               reversal_size,
               box_low,
               box_high):
    '''
    updates the chart once the trend status is defined
    returns status and box_range for the day
    
    Args:
        scale:
        high:
        low:
        status:
        reversal_size:
        box_low:
        box_high:
    Returns:
        status: int (-1,0, or 1)
        box_range: np.array
    '''
#     box_range = scale[np.logical_and(scale>=box_low, scale<=box_high)] # needed in case we return the current range
    box_range = generate_column_range(scale, box_low, box_high)
    box_reverse = []
    # new temporary box range with extensions on both sides:
#     box_range_new = scale[np.logical_and(scale>=min(low, box_low), scale<=max(high, box_high))]
    box_range_new = generate_column_range(scale, min(low, box_low), max(high, box_high))
    box_high_new = box_range_new.max()
    box_low_new = box_range_new.min()
    
    if status == 1:
        # check for upper extensions, else for reversals
        if box_high_new > box_high:
            box_range = scale[np.logical_and(scale>=box_low, scale<=box_high_new)]
#             use generate_...()
            # check for reversal
        elif low < box_high:
            box_reverse = scale[np.logical_and(scale>=low, scale<=box_high)][:-1]    
#             use generate_...()
            # research condition
            # change status and new range
                  
    if status == -1:
        if box_low_new < box_low:
            box_range = scale[np.logical_and(scale>=box_low_new, scale<=box_high)]
        elif high > box_low:
            box_reverse = scale[np.logical_and(scale>=box_low, scale<=high)][1:]

    # Check reversal and reverse status if needed:      
    if len(box_reverse) >= reversal_size:
        status *= -1 # reverse status
        box_range = box_reverse # update box_range
    
    return status, box_range

In [241]:
def pnf_text(scale, columns):
    '''
    Generates a text PnF chart
    
    Args:
        scale: np.array
        columns: list of tuples (int, np.array)
    Returns:
        grid: string
    '''
    hpad = 2 # padding columns on the sides
    marker = {0:'*', 1:'X', -1:'O'}
    grid = ""

    for line_price in np.flip(scale):
        line = f'{line_price}' + '.'*hpad
        for col in columns:
            line += marker[col[0]] if line_price in col[1] else '.'
        line += '.'*hpad + f'{line_price}\n'
        grid += line
    
    return grid

## Testing our functions

In [194]:
# testing init_pnf() on the 1st line of data:

box_size = 1
reversal_size = 3
scale = generate_scale(start=32, end=44, box_size = box_size)

high = 38.25
low = 35.75
close = 37.5

status, box_range = init_pnf(scale, high, low, close, reversal_size)
print(status, box_range)

1 [36. 37. 38.]


In [85]:
# testing init_pnf() on subsequent lines of data (when status==0)
# define: box_range array, status


high = 39.25
low = 35.01
close = 38

init_pnf(scale, high, low, close, reversal_size, box_range)

(1, array([36., 37., 38., 39.]))

In [104]:
# testing reversal logic

box_range = np.array([36, 37, 38])
status = 1
scale = scale
reversal_size = 3

high = 40.5
low = 35.5
box_low = box_range.min()
box_high = box_range.max()

box_reverse = []
# this can be simplified into one line using max()
# new range with extensions on both sides:
# if high > box_high:
#     box_range_new = scale[np.logical_and(scale>=box_low, scale<=high)]
# if low < box_low:
#     box_range_new = scale[np.logical_and(scale<=box_high, scale>=low)]
box_range_new = scale[np.logical_and(scale>=min(low, box_low), scale<=max(high, box_high))]

# we actually need only box_high_new and box_low_new
# box_high_new = box_range_new.max()
# box_low_new = box_range_new.min()

if status == 1:
    # we 1st check for extensions on the upside
    if box_range_new.max() > box_high:
        # extend range up:
        box_range = scale[np.logical_and(scale>=box_low, scale<=box_range_new.max())]
    # if no extension, check for reversal:
    elif low < box_high:
        box_reverse = scale[np.logical_and(scale>=low, scale<=box_high)][:-1]

if status == -1:
    # we 1st check for extensions on the downside
    if box_range_new.min() < box_low:
        # extend range down:
        box_range = scale[np.logical_and(scale>=box_range_new.min(), scale<=box_high)]
    # if no extension, check for reversal:
    elif high > box_low:
        box_reverse = scale[np.logical_and(scale>=box_low, scale<=high)][1:]

if len(box_reverse) >= reversal_size:
    status *= -1 # reverse status
    box_range = box_reverse
    
print(box_range, box_range_new, box_reverse, status)

[36. 37. 38. 39.] [36. 37. 38. 39.] [] 1


#### Tests for update_pnf() go here

In [16]:
# test 1
trend_status = np.array([1, 0, 0])
box_range = np.array([36, 37, 38])

reversal_size = 3

high = 40.12
low = 39.25

start = 1
status = trend_status[0]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(1, array([36, 37, 38, 39, 40]))

In [17]:
# test 2
trend_status = np.array([1, 1, 0])
box_range = np.array([36, 37, 38, 39, 40])

reversal_size = 3

high = 38.5
low = 37

status = trend_status[1]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(-1, array([37, 38, 39]))

In [18]:
# test 3
trend_status = np.array([1, 1, -1])
box_range = np.array([37, 38, 39])

reversal_size = 3

high = 40.25
low = 37

start = 1
status = trend_status[2]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(1, array([38, 39, 40]))

## Processing the data using the functions

In [227]:
price_data.head()

,High,Low,Close
0,401.940002,396.880005,401.679993
1,402.279999,393.290009,396.470001
2,395.890015,384.029999,384.290009
3,387.350006,380.130005,384.019989
4,385.760010,378.040009,379.859985


In [230]:
# initialise status and box arrays:
trend_status = np.zeros(len(price_data))
box_low = np.zeros(len(price_data))
box_high = np.zeros(len(price_data))

box_size = 10
reversal_size = 3
# scale = generate_scale(start=np.floor(price_data['Low'].min()), end=np.ceil(price_data['High'].max()), box_size=box_size)
scale = np.arange(260,430, 10)

print(f'Trend status: {trend_status}\nBox Low: {box_low}\nBox High: {box_high}\nScale: {scale}')

Trend status: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Box Low: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Box High: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
Scale: [260 270 280 290 300 310

In [231]:
# Initialise the chart until a status (+/-)1 is reached
box_range = []
for index, row in enumerate(price_data.iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = init_pnf(scale, high, low, close, reversal_size, box_range)
    trend_status[index] = status
    box_low[index] = box_range.min()
    box_high[index] = box_range.max()
    if status != 0:
        break

# status can still be 0! create an example for testing
print(f'Index: {index}\nTrend status: {trend_status}\nBox Low: {box_low}\nBox High: {box_high}\nRange: {scale[np.logical_and(scale>=box_low[index], scale<=box_high[index])]}')

Index: 4
Trend status: [ 0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
Box Low: [400. 400. 390. 390. 380.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.]
Box High: [400. 400. 400. 400. 400

In [232]:
# Check if there are more lines of data to process
print(index + 1 < len(price_data))

True


In [233]:
# Next, we need to process the prices after index:
start = index + 1
price_data.loc[start:].head()

,High,Low,Close
5,380.640015,369.309998,380.109985
6,385.989990,377.220001,385.820007
7,390.200012,385.010010,387.350006
8,389.470001,376.700012,377.660004
9,380.350006,374.859985,380.010010


In [234]:
# Process the remaining lines in price_data:
for index, row in enumerate(price_data.loc[start:].iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    status = trend_status[index + start - 1]
    box_l = box_low[index + start - 1]
    box_h = box_high[index + start - 1]
    status, box_range = update_pnf(scale, high, low, status, reversal_size, box_l, box_h)
    trend_status[index+start] = status
    box_low[index+start] = box_range.min()
    box_high[index+start] = box_range.max()
    print(f'Day: {index+start+1}, Trend status: {status}, High and Low: ', high, low, box_low, box_high)

# status can still be 0! create an example for testing that case
# print(index, trend_status, box_low, box_high)

Day: 6, Trend status: -1.0, High and Low:  380.640015 369.309998 [400. 400. 390. 390. 380. 370.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.] [400. 400. 400. 400. 400. 400.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

Day: 79, Trend status: -1.0, High and Low:  327.660004 316.859985 [400. 400. 390. 390. 380. 370. 370. 370. 370. 370. 370. 370. 370. 360.
 340. 340. 340. 340. 340. 340. 340. 350. 350. 350. 350. 350. 350. 350.
 350. 350. 350. 350. 350. 340. 340. 330. 320. 320. 320. 320. 320. 330.
 330. 330. 320. 320. 320. 320. 320. 320. 320. 320. 330. 330. 330. 330.
 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 340.
 340. 340. 340. 340. 340. 340. 330. 330. 320.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.] [400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400.
 400. 400. 400. 400. 400. 400. 400. 370. 370. 370. 370. 370. 370. 370.
 370. 370. 370. 370. 370. 360. 360. 360. 360. 360. 360. 360. 360. 350.
 350. 350. 340. 340. 340. 340. 340. 340. 340. 340. 350. 350. 350. 350.
 350. 360. 360. 370. 370. 370. 370. 370. 370. 370. 370. 370. 370. 360.
 360. 360. 360. 360. 360. 360. 360. 360. 360.   0.   0.   0.

In [235]:
print(trend_status, box_low, box_high)

[ 0.  0.  0.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1.  1.  1.  1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.] [400. 400. 390. 390. 380. 370. 370. 370. 370. 370. 370. 370. 370. 360.
 340. 340. 340. 340. 340. 340. 340. 350. 350. 350. 350. 350. 350. 350.
 350. 350. 350. 350. 350. 340. 340. 330. 320. 320. 320. 320. 320. 330.
 330. 330. 320. 320. 320. 320. 320. 320. 320. 320. 330. 330. 330. 330.
 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 330. 340.
 340. 340. 340. 340. 340. 340. 330. 330. 320. 320. 320. 320. 310. 310.
 310. 310. 310. 300. 300. 300. 290. 290. 290. 290. 290. 290. 290. 290.
 290. 290. 290. 290.] [400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 400. 

## Printing the PnF chart

In [236]:
trend_status = trend_status
box_low = box_low
box_low = box_low

pnf_data = pd.DataFrame({'trend_status': trend_status,
                         'range_low': box_low,
                         'range_high': box_high
                        })
pnf_data

,trend_status,range_low,range_high
0,0.0,400.0,400.0
1,0.0,400.0,400.0
2,0.0,390.0,400.0
3,0.0,390.0,400.0
4,-1.0,380.0,400.0
...,...,...,...
97,-1.0,290.0,360.0
98,-1.0,290.0,360.0
99,-1.0,290.0,360.0
100,-1.0,290.0,360.0


We plot a column each time the **change** bool array is True (note: it's shifted up), then for the last row

In [237]:
changes = (np.diff(np.sign(trend_status)) != 0)
# We make sure that the a column is generated for the last price line:
changes = np.append(changes, [True])
changes

array([False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True, False, False,  True, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True])

In [238]:
# Note that the change column is 'shifted': it's True when a status change is detected on the next price line:
pnf_data['change'] = changes
pnf_data

,trend_status,range_low,range_high,change
0,0.0,400.0,400.0,False
1,0.0,400.0,400.0,False
2,0.0,390.0,400.0,False
3,0.0,390.0,400.0,True
4,-1.0,380.0,400.0,False
...,...,...,...,...
97,-1.0,290.0,360.0,False
98,-1.0,290.0,360.0,False
99,-1.0,290.0,360.0,False
100,-1.0,290.0,360.0,False


In [239]:
ranges = []
trends = []
scale = scale

# should we use .apply() here?
for row in pnf_data[pnf_data['change']].iterrows():
    row = row[1]
    col_range = generate_column_range(scale, row['range_low'], row['range_high'])
    ranges.append(col_range)
    trends.append(row['trend_status'])

columns = list(zip(trends, ranges))

print(columns[:10])

[(0.0, array([390, 400])), (-1.0, array([340, 350, 360, 370, 380, 390, 400])), (1.0, array([350, 360, 370])), (-1.0, array([320, 330, 340, 350, 360])), (1.0, array([330, 340, 350])), (-1.0, array([320, 330, 340])), (1.0, array([330, 340, 350, 360, 370])), (-1.0, array([290, 300, 310, 320, 330, 340, 350, 360]))]


In [242]:
print(pnf_text(scale, columns))

420............420
410............410
400..*O........400
390..*O........390
380...O........380
370...OX...X...370
360...OXO..XO..360
350...OXOX.XO..350
340...O.OXOXO..340
330.....OXOXO..330
320.....O.O.O..320
310.........O..310
300.........O..300
290.........O..290
280............280
270............270
260............260

